In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np

import datajoint as dj

In [ ]:
acq = dj.create_virtual_module('acq', 'data_acq')

In [ ]:
dj.ERD(acq)

In [ ]:
dj.ERD(acq) + 2

In [ ]:
detect = dj.create_virtual_module('detect', 'data_detect')

In [ ]:
dj.ERD(detect) + 2

In [ ]:
sort = dj.create_virtual_module('sort', 'data_sort')

In [ ]:
dj.ERD(sort)+2

In [ ]:
ephys = dj.create_virtual_module('ephys', 'data_ephys')

In [ ]:
dj.ERD(ephys)-1

In [ ]:
dj.ERD(acq) + dj.ERD(ephys) + dj.ERD(sort) + dj.ERD(detect) + 3 - 3 + 3 -3

In [ ]:
ephys.Spikes()

In [ ]:
acq.Subjects.aggr(ephys.Spikes, n = 'count(*)')

In [ ]:
acq.Sessions.aggr(ephys.Spikes, n = 'count(*)')

In [ ]:
ephys.Spikes().describe();

In [ ]:
keys = ephys.Spikes.fetch('KEY')

In [ ]:
key = keys[125]

In [ ]:
ephys.Spikes & key

In [ ]:
acq.Sessions & key

In [ ]:
# other spike trains from the same session
other_spikes = (ephys.Spikes & (acq.Sessions & key)) - key

In [ ]:
other_spikes

In [ ]:
spike_times, waveform = (ephys.Spikes & key).fetch1('spike_times', 'mean_waveform', squeeze=True)

In [ ]:
plt.plot(np.hstack(other_spikes.fetch('mean_waveform')), 'k', lw=0.25, alpha=0.5)
plt.plot(waveform, 'r')
plt.axis('off');

In [ ]:
plt.hist(np.diff(spike_times), 30);

In [ ]:
for tup in ephys.Spikes & (acq.Sessions & key):
    print('Unit: ', tup['unit_id'], 'Mean ISI', np.diff(tup['spike_times'].flatten()).mean())

In [ ]:
schema = dj.schema('dimitri_spikes')    # change to your username prefix

In [ ]:
@schema
class SpikeStats(dj.Computed):
    definition = """
    -> ephys.Spikes
    ---
    mean_isi : float  # (s)
    """
    def make(self, key):
        spike_times = (ephys.Spikes & key).fetch1('spike_times', squeeze=True)
        key['mean_isi'] = np.diff(spike_times).mean()
        self.insert1(key)

In [ ]:
dj.ERD(SpikeStats)-2

In [ ]:
SpikeStats.populate(display_progress=True)

In [ ]:
SpikeStats.progress();

In [ ]:
acq.Sessions.aggr(SpikeStats, mean_isi='avg(mean_isi)')